# Vectorisation 
## X-vectors

pip install speechbrain

In [41]:
# librairies 
import glob 
import speechbrain as sb
import pandas as pd
import torch
import os
import torchaudio
from speechbrain.inference.speaker import EncoderClassifier


In [20]:
# Ouverture fichiers

chemin_base = "./data" # VERSION GIT 

list_fichier = glob.glob(f"{chemin_base}/ESLO2_*/*.wav")
print(list_fichier)

metadata_fichier = pd.read_csv("./data/metadonnees_ESLO2_ENT_ENTJEUN.csv")

['./data/ESLO2_ENT_1001/ESLO2_ENT_1001_0.wav', './data/ESLO2_ENT_1001/ESLO2_ENT_1001_2.wav', './data/ESLO2_ENT_1001/ESLO2_ENT_1001_1.wav']


In [33]:
# rajout classses d'age
def class_age(age):
	if age == 'vieux' or int(age) > 60: 
		return "vieux"
	elif int(age) < 30:
		return "jeune"
	else:
		return "mid"
	
metadata_fichier['classe_age'] = metadata_fichier['age'].apply(class_age)

In [30]:
# vectorisation 
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

def vectorisation(fichier):
	signal, fs = torchaudio.load(fichier)
	embeddings = classifier.encode_batch(signal)
	return embeddings


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/home/tacitus/Documents/M2_TAL/ML/.machinelearn

In [42]:
# enregistrement des vecteurs dans leur bon directory (selon classe d'age et sexe) 
# VERSION GIT 

for fichier in list_fichier: 
	embeddings = vectorisation(fichier)
	if fichier.split("/")[-2] in metadata_fichier['directory'].values:
		age = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['classe_age'].values[0]
		sexe = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['sexe'].values[0]
	if not os.path.exists(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}"):
		os.makedirs(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}")
	chemin = f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}/{fichier.split('/')[-1].replace('.wav', '.pt')}"
	
	torch.save(embeddings, chemin)


In [ ]:
# enregistrement des vecteurs dans leur bon directory (selon classe d'age et sexe) 
# VERSION uniquement HOMME sur GIT  

for fichier in list_fichier: 
	embeddings = vectorisation(fichier)
	if fichier.split("/")[-2] in metadata_fichier['directory'].values:
		age = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['classe_age'].values[0]
		sexe = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['sexe'].values[0]
	if sexe == "homme":
		if not os.path.exists(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}"):
			os.makedirs(f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}")
		chemin = f"./vectors/{sexe}/{age}/{fichier.split('/')[-2]}/{fichier.split('/')[-1].replace('.wav', '.pt')}"
		
		torch.save(embeddings, chemin)
	else: 
		continue


In [ ]:
# enregistrement des vecteurs dans leur bon directory (selon classe d'age et sexe) 
# VERSION DISQUE DUR  

base_disque = ""

for fichier in list_fichier: 
	embeddings = vectorisation(fichier)
	if fichier.split("/")[-2] in metadata_fichier['directory'].values:
		age = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['classe_age'].values[0]
		sexe = metadata_fichier[metadata_fichier['directory'] == fichier.split("/")[-2]]['sexe'].values[0]
	if not os.path.exists(f".{base_disque}/vectors/{sexe}/{age}/{fichier.split('/')[-2]}"):
		os.makedirs(f".{base_disque}/vectors/{sexe}/{age}/{fichier.split('/')[-2]}")
	chemin = f".{base_disque}/vectors/{sexe}/{age}/{fichier.split('/')[-2]}/{fichier.split('/')[-1].replace('.wav', '.pt')}"
	
	torch.save(embeddings, chemin)
